In [1]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch

2025-02-19 23:42:22.652937: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 23:42:22.666653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-19 23:42:22.683999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-19 23:42:22.689417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 23:42:22.701852: I tensorflow/core/platform/cpu_feature_guar

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring.

In [2]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

In [4]:
adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

/opt/conda/lib/python3.11/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'model_type', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/opt/conda/lib/python3.11/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['model_type'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 67.97 out of 124.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:00<00:01, 15.85it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:14<00:00,  2.25it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [8]:
adapter_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [9]:
!ls

'Finetuning Story Generation Model.ipynb'   hf_auth.ipynb
 README.md				    inference_container
 Story-Generation-LlaMA-3.1-8B-10k	    load_data.ipynb
 feature_engineering.ipynb		    unsloth_compiled_cache
 format_pth_model.ipynb


## Saving the Merged Model 
The model will be pushed to the INFER_MODEL huggingface repo, and will also be saved as a model .pth file for use in the container for inferrence.

In [3]:
pth_model = AutoModelForCausalLM.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k", torchscript=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k", torchscript=True)

In [12]:
pth_model.push_to_hub(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00001-of-00012.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00002-of-00012.safetensors:   0%|          | 0.00/2.79G [00:00<?, ?B/s]

model-00004-of-00012.safetensors:   0%|          | 0.00/2.85G [00:00<?, ?B/s]

Upload 12 LFS files:   0%|          | 0/12 [00:00<?, ?it/s]

...

model-00009-of-00012.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00010-of-00012.safetensors:   0%|          | 0.00/2.85G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/commit/88ffdff929c9a8b309dc5464bf1d9e1873cfb75d', commit_message='Upload LlamaForCausalLM', commit_description='', oid='88ffdff929c9a8b309dc5464bf1d9e1873cfb75d', pr_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/discussions/2', repo_url=RepoUrl('https://huggingface.co/Alexis-Az/Story-Generation-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Alexis-Az/Story-Generation-Model'), pr_revision='refs/pr/2', pr_num=2)